# Midterm 1 Assignment 6

#### Various Imports

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
def show_img(image: np.ndarray, gray: bool = True) -> None:
    """simple utility to show images with axes disabled

    Parameters
    ----------
    image : np.ndarray
        image to show
    gray : bool, optional
        if true show image in grayscale, by default True
    """
    if gray:
        plt.imshow(image, cmap="gray")
    else:
        plt.imshow(image)
    plt.axis("off")
    plt.show()

# Implementation of necessary methods

### Convolution Method

Padding for boundaries of image is handled by replicating the nearest element of the image by default. This can be changed to any type of padding supported by OpenCV

In [ ]:
def img_filter_convolution(image: np.ndarray, filter: np.ndarray, padtype=cv2.BORDER_REPLICATE) -> np.ndarray:
    """function to convolve an image with a filter

    Parameters
    ----------
    image : np.ndarray
        image to convolve
    filter : np.ndarray
        filter to convolve
    padtype : _type_, optional
        type of padding to use, by default cv2.BORDER_REPLICATE

    Returns
    -------
    np.ndarray
        result of convolution of image with filter
    """
    x_size, y_size = image.shape
    filter_size = filter.shape[0]
    padding = filter_size//2
    padded_image = cv2.copyMakeBorder(image, padding, padding, padding, padding, padtype)
    filtered_img = np.zeros((x_size, y_size))

    for x in range(x_size):
        for y in range(y_size):
            filtered_img[x, y] = (padded_image[x: x+filter_size, y:y+filter_size] * filter).sum()
    return filtered_img

### Gaussian Filter Implementation
The filter is created with using a radius $r$ of $\lceil 3 \sigma \rceil$. This gives a filter of size $(2r + 1)\times(2r + 1)$. Both dimensions, being the same, are always odd, allowing easy convolution of the filter with an image.<br>
This size was picked because the values of the filter become very small after that point, thus making it a good cutoff for a discrete approximation of a Gaussian filter.

In [ ]:
def gaussian_filter(scale: float, size: int = None) -> np.ndarray:
    """function returning a gaussian filter at given scale

    Parameters
    ----------
    scale : float
        scale to use for the gaussian filter
    size : int, optional
        size of filter, by default None

    Returns
    -------
    np.ndarray
        the gaussian filter at given scale and either given size or computed one
    """
    #if size of the filter is not given, use ceiling of 3*scale as radius r, giving a filter of size (2*r + 1)x(2*r +1)
    if size is None:
        size = int(2 * np.ceil(3*scale)) + 1
    v = np.arange((-size // 2) + 1, (size // 2) + 1)
    x = v * np.ones((size, size))
    y = x.T
    filter = 1/(2*np.pi*scale**2) * np.exp(-(x*x + y*y)/(2*scale**2))
    return filter

### LoG Filter Implementation

To determine the size of the filter the same method as the Gaussian filter was used.

In [ ]:
def LoG(scale: float, size:int = None) -> np.ndarray:
    """function returning a laplacian of gaussian filter at given scale

    Parameters
    ----------
    scale : float
        scale to use for the gaussian filter
    size : int, optional
        size of filter, by default None

    Returns
    -------
    np.ndarray
        the LoG filter at given scale and either given size or computed one
    """
    if size is None:
        size = int(2 * np.ceil(3*scale)) + 1
    v = np.arange((-size // 2) + 1, (size // 2) + 1)
    x = v * np.ones((size, size))
    y = x.T
    gaussian = 1/(2*np.pi*scale**2) * np.exp(-(x*x + y*y)/(2*scale**2))
    log_filter = ((x*x + y*y)/(scale**4) - 2/(scale**2))*gaussian
    return log_filter


### DoG Filter Implementation

In [ ]:
def DoG(scale: float, size:int = None, k=2) -> np.ndarray:
    pass

### Blob Detection Implementation

In [ ]:
def blob_detector(image: np.ndarray, scales:list[int], threshold:float = 0.03) -> list[tuple[int, int, float]]:
    """function implementing a scale space blob detector

    Parameters
    ----------
    image : np.ndarray
        image to run blob detection on
    scales : list[int]
        list of scales to use
    threshold : float, optional
        a point that is a potential center of a blob must be greater than this value to be actually picked, by default 0.03

    Returns
    -------
    list[tuple[int, int, float]]
        a list containing tuples where the first two elements are coordinates and the third is the scale at which that point was chosen.
        Each tuple is the center of a blob at that scale
    """
    outputs = []
    img_x, img_y = image.shape
    for scale in scales:
        filter = LoG(scale)
        output = scale * scale * img_filter_convolution(image, filter)
        output = output * output
        output = cv2.copyMakeBorder(output, 1, 1, 1, 1, cv2.BORDER_CONSTANT, 0)
        outputs.append(output)
    outputs = np.asarray(outputs)
    
    centers = []
    test_list = []
    for x in range(1, img_x+1):
        for y in range(1, img_y+1):
            current_slice = outputs[:,x-1:x+2,y-1:y+2]
            test_list.append(np.amax(current_slice))
            if np.amax(current_slice) > threshold:
                indices = np.unravel_index(np.argmax(current_slice), current_slice.shape)
                if indices[1] == 1 and indices[2] == 1:
                    centers.append((x, y, scales[indices[0]]))
    return centers


## Tests for LoG filter and convolution

In [ ]:
image = cv2.imread("./MSRC_ObjCategImageDatabase_v1/3_22_s.bmp")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image normalizion and moving to grayspace instead of RGB
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) / 255.0
show_img(gray)

In [ ]:
gauss = gaussian_filter(5)
blurred_img = img_filter_convolution(gray, gauss)
show_img(gauss)
show_img(blurred_img)

In [ ]:
laplacian = LoG(5)
laplacian_img = img_filter_convolution(gray, laplacian)

show_img(laplacian)
show_img(laplacian_img)

## Blob detection on selected images

### Image 1

### Image 2

In [ ]:
image = cv2.imread("./MSRC_ObjCategImageDatabase_v1/3_22_s.bmp")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) / 255.0
outputs = []
scales = [1, 5, 10, 15, 20, 30]
results = blob_detector(gray, scales, 0.03)


In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image)
for x, y, scale in results:
    c = plt.Circle((y, x), scale*np.sqrt(2), color="red", linewidth = 1.5, fill=False)
    ax.add_patch(c)
ax.plot()
plt.show()

## Workflow for a single image

In [ ]:
image = cv2.imread("/home/davide/uni/ISPR-Midterms/Midterm1/MSRC_ObjCategImageDatabase_v1/1_11_s.bmp")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

plt.imshow(gray, cmap="gray")
plt.show()

Run blob detection on the image at a given scale

In [ ]:
scale = 5
tmp = blob_detection(gray, scale)
plt.imshow(tmp, cmap="gray")
plt.show()

# Image 1

In [ ]:
image = cv2.imread("/home/davide/uni/ISPR-Midterms/Midterm1/MSRC_ObjCategImageDatabase_v1/sunflowers.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

outputs = []
scales = [1, 2]
blob_detector(gray, scales)


In [ ]:
centers = []
for output, scale in outputs:
    ret, thresholded_output = cv2.threshold(output, 150, 255, cv2.THRESH_BINARY)
    thresholded_output = thresholded_output.astype(np.uint8)
    contours = cv2.findContours(thresholded_output,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
    for i in contours:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            centers.append((cx, cy, scale))
        # print(f"x: {cx} y: {cy} scale: {scale}")

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image)
for x, y, scale in centers:
    c = plt.Circle((x, y), scale*np.sqrt(2), color="red", linewidth = 1.5, fill=False)
    ax.add_patch(c)
# x, y = output.shape
# for row in range(x):
#     for column in range(y):
#         if thresholded_output[row, column] != 0:
#             c = plt.Circle((column, row), scale*np.sqrt(2), color="red", linewidth = 0.5, fill=False)
#             ax.add_patch(c)
ax.plot()
plt.show()